# Problema dos missionários e canibais

---

**Implementado com algoritmo de Holland**

---

Objetivo: mover três missionários e três canibais de um lado do rio para o outro, usando um barco que só pode transportar uma ou duas pessoas por vez. Em qualquer lado do rio, o número de missionários não pode ser menor que o número de canibais, caso contrário os canibais comerão os missionários.


**Codificação do cromossomo**:
```
individuo = [p1,p2,p3,p4,p5,p6]
```
A lista contém 6 posições que representam as 6 pessoas do problema: os 3 missionários e os 3 canibais, respectivamente.

>  pᵢ = 0, indica que a pessoa está na margem esquerda;

>  pᵢ = 1, indica que a pessoa está na margem direita.


Deseja-se alcançar o indivíduo com o seguinte cromossomo:
```
individuo_alvo = [1,1,1,1,1,1]
```

In [1]:
from random import randint

**População inicial**

In [2]:
# Gera aleatoriamente uma população de indivíduos
def inicia_populacao(tam):
    return [[randint(0,1) for i in range(6)] for j in range(tam)]

**Cálculo de adaptação**

*calcula_adaptacao* usa a função de aptidão abaixo:
```
adap(individuo) = 𝛴 individuo[i]
```
Como a solução é representada pelo cromossomo``` [1, 1, 1, 1, 1, 1],``` procura-se maximizar ```𝛴 individuo[i].```

Obviamente, há punição para indivíduos *monstros.* Um indivíduo é considerado *monstro,* quando seu cromossomo representa uma situação inválida no problema real. Ou seja, quando há mais canibais que missionários em alguma margem. Esse tipo de indivíduo recebe adaptação negativa, visto que está fora do escopo do problema. Assim impede-se sua permanência na população por meio de seleção.

In [3]:
def calcula_adaptacao(individuo):

    missionarios = individuo[:3]
    canibais = individuo[3:]

    # missionarios na margem direita
    md = sum(missionarios)
    # canibais na margem direita
    cd = sum(canibais)
    # missionarios na margem esquerda
    me = 3 - md
    # canibais na margem esquerda
    ce = 3 - cd

    # verifica se há mais canibais que missionários em alguma margem
    if (ce > me and me > 0) or (cd > md and md > 0):
        return -1

    return sum(individuo)

**Seleção dos mais adaptados**

In [4]:
# Ordena a população do mais adaptado ao menos adaptado
def ordena(pop, adap):

    tam = len(pop)
    for i in range(tam):
        for j in range(tam-1, i, -1):
            if adap[i] < adap[j]:
                adap[i], adap[j] = adap[j], adap[i]
                pop[i], pop[j] = pop[j], pop[i]

In [5]:
# Escolhe os 5 indivíduos mais adaptados da população
def seleciona(populacao):

    adaptacoes = [calcula_adaptacao(ind) for ind in populacao]
    ordena(populacao, adaptacoes)
    del populacao[5:]

**Cruzamento**

*cruza* gera filhos a partir da combinação de dois indivíduos (cromossomos) após a seleção dos mais adaptados. Essa função é aplicada em todos os pares da população.

Para aumentar a diversidade genética vamos usar dois pontos de cortes aleatórios r1 e r2.

In [6]:
def cruza(populacao):

    filhos = []
    for ind1 in populacao:
        for ind2 in populacao:
            if ind1 != ind2:
                r1 = randint(1,5)
                r2 = randint(1,5)
                # ordena os pontos de corte
                r1, r2 = min(r1,r2), max(r1,r2)
                f1 = ind1[:r1] + ind2[r1:r2] + ind1[r2:]
                f2 = ind2[:r1] + ind1[r1:r2] + ind2[r2:]
                if f1 not in filhos and f1 not in populacao:
                    filhos.append(f1)
                if f2 not in filhos and f2 not in populacao:
                    filhos.append(f2)
    populacao += filhos

**Mutação**

In [7]:
def muta(populacao):

    for individuo in populacao:
        r = randint(0,100)
        if r >= 90:
            for i in range(6):
                r = randint(0,1)
                if r == 1:
                    individuo[i] = int(not individuo[i])

---

**Execução**

In [8]:
solucao = [1,1,1,1,1,1]

In [9]:
# Gerar uma população inicial com 10 indivíduos
populacao = inicia_populacao(10)
populacao

[[1, 0, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 1, 1],
 [0, 1, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 0],
 [0, 1, 1, 0, 1, 1],
 [1, 0, 1, 1, 0, 0],
 [0, 0, 1, 0, 1, 0]]

In [10]:
# Verificar as adaptações dos individuos iniciais
[calcula_adaptacao(ind) for ind in populacao]

[2, -1, -1, 2, -1, -1, -1, 4, -1, 2]

In [11]:
seleciona(populacao)
populacao

[[0, 1, 1, 0, 1, 1],
 [0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 1, 1]]

In [12]:
while solucao not in populacao:
    cruza(populacao)
    muta(populacao)
    seleciona(populacao)
    print(f'População pós seleção: {populacao}\n')

População pós seleção: [[0, 1, 1, 0, 1, 1], [1, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1], [0, 0, 0, 1, 1, 1]]

População pós seleção: [[1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1], [0, 1, 1, 0, 1, 1]]

População pós seleção: [[1, 1, 1, 0, 1, 1], [0, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 0], [1, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0]]

População pós seleção: [[1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 0], [1, 0, 1, 0, 1, 1], [1, 1, 1, 1, 0, 0]]

População pós seleção: [[1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 0], [1, 0, 1, 0, 1, 1], [1, 1, 1, 1, 0, 0]]

